In [21]:
import tensorflow as tf

# Load the data

In [22]:
import os


data_dir = os.path.abspath('/mnt/f/Courses/4GeeksAcademy/DataScience/DeliverableExercises/alejandroNovellino-image-classifier-project/data/raw/test')

# set the image width and height
img_width = 160
img_height = 160

batch_size = 16

# train dataset
test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    labels='inferred',
    label_mode='categorical'
)

Found 140 files belonging to 2 classes.


In [23]:
test_ds

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 2), dtype=tf.float32, name=None))>

In [24]:
class_names = test_ds.class_names
print(class_names)

['cat', 'dog']


We have two labels the **cat** and **dog** labels.

# Configure the dataset for performance

We are going to use buffered prefetching, so we can yield data from disk without having I/O become blocking. The two important methods we should use when loading data are:

- **Dataset.cache** keeps the images in memory after they're loaded off disk during the first epoch. This will ensure the dataset does not become a bottleneck while training your model. If your dataset is too large to fit into memory, you can also use this method to create a performant on-disk cache.
- **Dataset.prefetch** overlaps data preprocessing and model execution while training.
Interested readers can learn more about both methods, as well as how to cache data to disk in the Prefetching section of the Better performance with the tf.data API guide.

In [25]:
train_ds = test_ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

The model itself does the normalization, so no work on the data must be done.

# Now let's load the model

In [26]:
import keras

model_dir = os.path.abspath('/mnt/f/Courses/4GeeksAcademy/DataScience/DeliverableExercises/alejandroNovellino-image-classifier-project/models/base_model.keras')

model = keras.saving.load_model(model_dir)

# Analyze the results

In [27]:
# make the prediction
y_pred = model.predict(test_ds, verbose=0)

In [28]:
y_pred

array([[0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [0.5010699, 0.4989301],
       [

In [29]:
import pandas as pd

# create a dataframe with the predictions
y_pred_df = pd.DataFrame(y_pred)
y_pred_df['predicted_class'] = y_pred_df.idxmax(axis=1)
y_pred_df['predicted_proba'] = y_pred_df[[0, 1]].max(axis=1)

In [30]:
y_pred_df

,0,1,predicted_class,predicted_proba
0,0.50107,0.49893,0,0.50107
1,0.50107,0.49893,0,0.50107
2,0.50107,0.49893,0,0.50107
3,0.50107,0.49893,0,0.50107
4,0.50107,0.49893,0,0.50107
...,...,...,...,...
135,0.50107,0.49893,0,0.50107
136,0.50107,0.49893,0,0.50107
137,0.50107,0.49893,0,0.50107
138,0.50107,0.49893,0,0.50107


# Metrics

We can calculate the accuracy of the CNN with the model.evaluate method. Or/and we can see the results wht sckit

In [31]:
_, accuracy = model.evaluate(test_ds)

print(f"Test accuracy: {round(accuracy * 100, 2)}%")

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 77ms/step - accuracy: 0.4998 - loss: 0.6932
Test accuracy: 50.0%


# Clear the memory

In [32]:
keras.backend.clear_session()